<a href="https://colab.research.google.com/github/ane-schow/oceanografia/blob/main/pcolourmesh%20com%20contour%20foz%20amazonas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gráfico de temperatura visto de cima na foz do amazonas com contorno

In [18]:
!sudo apt-get update -y
!sudo apt-get install python3.11
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1
!sudo apt-get install python3.11-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python3.11 get-pip.py
!python3 --version
!python3.11 -m pip install cartopy

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,654 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,450 kB]
Ign:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]


In [19]:

import xarray as xr
from IPython.display import display
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import numpy as np

In [ ]:
fn = '/content/thetao.nc'
dataset = xr.open_dataset(fn)
dataset.info()

Separar variáveis

In [ ]:
lat = dataset['latitude']
long = dataset['longitude']
profundidade = dataset['depth']
idx = profundidade.where(profundidade<=50, drop=True)
print(idx)
T = dataset['thetao'].isel(time=slice(0,33), depth=slice(0, 51), latitude=slice(0, 223), longitude=slice(0, 214)).mean(dim='time')
t_media = T.isel(depth=slice(0,19), latitude=slice(0, 223), longitude=slice(0, 214)).mean(dim='depth')
display(t_media)

Plotar o gráfico

In [35]:
temperatura_media = t_media
latitude = t_media['latitude']
longitude = t_media['longitude']

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())

pca = ax.pcolormesh(longitude,latitude,temperatura_media,transform=ccrs.PlateCarree(),cmap='plasma')
cntr = ax.contour(longitude, latitude, temperatura_media, levels=17, transform=ccrs.PlateCarree(),linewidth=0.5, colors='w',alpha=0.8)
ax.clabel(cntr, levels=cntr.levels[::3], colors='w', fontsize=8)

c = plt.colorbar(pca, ax=ax)
c.set_label('Temperatura média (C)')
ax.add_feature(cfeature.LAND, color='k', alpha=0.8)
ax.add_feature(cfeature.RIVERS.with_scale('10m'), edgecolor='aqua',alpha=0.65)

gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True, linewidth=0.5,color='gray',alpha=0.7, linestyle=':')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
ax.set_title("Temperatura média da Foz do amazonas no período de 1 mês nos primeiros 50 metros de coluna d'água")
plt.show()